In [ ]:
# Federated Learning - Section Project #2 - Toy Database
# Central server is not trusted with dataset

# code sources:
# https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%204%20-%20Federated%20Learning%20via%20Trusted%20Aggregator.ipynb

'''
1. Train a dataset
2. Use federated learning
3. Gradients not come up to central learning in raw form
4. Use .move command to move gradients to one of the workers
5. Sum gradients at one worker 
6. Bring sum back to central server
Note: central server never sees gradient for one worker - one worker does see gradient
'''

In [1]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

W0728 18:24:18.930500  5252 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Claudia\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0728 18:24:18.951499  5252 deprecation_wrapper.py:119] From C:\Users\Claudia\Anaconda3\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
# Orchestrate movement of data from one virtual machine to another
# worker simulates interface we have to another machine
# create data ownders as a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [3]:
# Create model - Initalize A Toy Model
model = nn.Linear(2,1)

In [4]:
# send copy of model to alice and bob

bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [5]:
#train bob and alice's model in parallel

for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.1160) Alice:tensor(0.9489)
Bob:tensor(0.0776) Alice:tensor(0.0190)
Bob:tensor(0.0614) Alice:tensor(0.0099)
Bob:tensor(0.0513) Alice:tensor(0.0082)
Bob:tensor(0.0435) Alice:tensor(0.0068)
Bob:tensor(0.0370) Alice:tensor(0.0057)
Bob:tensor(0.0316) Alice:tensor(0.0047)
Bob:tensor(0.0269) Alice:tensor(0.0039)
Bob:tensor(0.0230) Alice:tensor(0.0033)
Bob:tensor(0.0196) Alice:tensor(0.0027)


In [6]:
# send both models to a secure worker
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [8]:
#average trained models together
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [9]:
# iterate a couple of times
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0178) Alice:tensor(0.0001)
Bob:tensor(0.0126) Alice:tensor(6.8235e-06)
Bob:tensor(0.0090) Alice:tensor(1.9779e-06)
Bob:tensor(0.0066) Alice:tensor(1.2777e-05)
Bob:tensor(0.0049) Alice:tensor(2.1747e-05)
Bob:tensor(0.0036) Alice:tensor(2.5947e-05)
Bob:tensor(0.0027) Alice:tensor(2.6294e-05)
Bob:tensor(0.0021) Alice:tensor(2.4323e-05)
Bob:tensor(0.0016) Alice:tensor(2.1266e-05)
Bob:tensor(0.0012) Alice:tensor(1.7917e-05)


In [10]:
#evalutate dataset
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [11]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0957],
        [0.0774],
        [0.9027],
        [0.8844]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0380)
